*Import necessary Modules*

In [14]:
import os
import random
import shutil
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt
from PIL import Image

*Mount drive for a dataset*

In [15]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Extract RAR to folder*

In [16]:
rar_path = '/content/drive/My Drive/CUDA-dataset/TrashNet.rar'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

!unrar x -o+ "{rar_path}" "{extract_path}"

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Extracting  /content/dataset/TrashNet/metal/metal_containers 259.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 260.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 264.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 266.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 267.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 275.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 279.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 282.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 284.jpg      45%  OK 
Extracting  /content/dataset/TrashNet/metal/metal_containers 294.jpg      45%  OK 
Extracting  /co

*Split data to Train and Validation sets*

In [17]:
#shutil.rmtree('dataset/TrashNet/train')
#shutil.rmtree('dataset/TrashNet/validation')
base_dir = 'dataset/TrashNet'
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError) as e:
        return False

def convert_image(file_path):
    try:
        with Image.open(file_path) as img:
            if img.mode in ("P", "RGBA"):
                img = img.convert("RGBA")
            else:
                img = img.convert("RGB")
            img.save(file_path)
    except (IOError, SyntaxError) as e:
        os.remove(file_path)

for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        if filename.lower().endswith(('.jpg', '.JPG')):
            if is_valid_image(file_path):
                convert_image(file_path)
            else:
                os.remove(file_path)
        else:
            os.remove(file_path)

split_ratio = 0.8

for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    images = os.listdir(class_dir)
    random.shuffle(images)

    split_point = int(split_ratio * len(images))
    train_images = images[:split_point]
    validation_images = images[split_point:]

    for img in train_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(train_dir, cls, img)
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        shutil.move(source, destination)

    for img in validation_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(validation_dir, cls, img)
        os.makedirs(os.path.join(validation_dir, cls), exist_ok=True)
        if img not in train_images:  # Sprawdź, czy obraz nie został już użyty w zbiorze treningowym
            shutil.move(source, destination)

*Removing existing directories - after splitting*

In [18]:
base_path = '/content/dataset/TrashNet'
for cls in classes:
    folder_path = os.path.join(base_path, cls)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

*Prepare Data Generators*

In [19]:
# Ustawienia
img_height = 224
img_width = 224
batch_size = 32
epochs = 5

In [20]:
# Przygotowanie danych
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 8837 images belonging to 6 classes.
Found 3302 images belonging to 6 classes.


*Creation of a simple CNN Model*

In [22]:
# Tworzenie modelu
num_classes = len(train_generator.class_indices)

model_simpleCNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


In [23]:
model_simpleCNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_simpleCNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 26, 26, 128)      

In [24]:
history_simpleCNN = model_simpleCNN.fit(
    # provides batches of samples to train on each epoch
    train_generator,

    # train_generator.samples - total number of training samples
    # it calculates the number of steps needed to process the entire training data sets once in each epoch
    steps_per_epoch=train_generator.samples // batch_size,

    epochs=15,

    #  the Model will evaluate the performance on this validation data
    validation_data=validation_generator,

    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/15
276/276 [==============================] - 165s 581ms/step - loss: 1.5411 - accuracy: 0.4083 - val_loss: 1.3947 - val_accuracy: 0.4675
Epoch 2/15
276/276 [==============================] - 161s 583ms/step - loss: 1.3076 - accuracy: 0.4868 - val_loss: 1.2420 - val_accuracy: 0.4848
Epoch 3/15
276/276 [==============================] - 161s 582ms/step - loss: 1.1835 - accuracy: 0.5491 - val_loss: 1.0576 - val_accuracy: 0.5995
Epoch 4/15
276/276 [==============================] - 161s 583ms/step - loss: 1.0749 - accuracy: 0.5945 - val_loss: 1.0236 - val_accuracy: 0.6262
Epoch 5/15
276/276 [==============================] - 154s 555ms/step - loss: 1.0141 - accuracy: 0.6209 - val_loss: 0.8997 - val_accuracy: 0.6899
Epoch 6/15
276/276 [==============================] - 157s 570ms/step - loss: 0.9580 - accuracy: 0.6465 - val_loss: 0.9888 - val_accuracy: 0.6047
Epoch 7/15
276/276 [==============================] - 154s 560ms/step - loss: 0.9221 - accuracy: 0.6610 - val_loss: 0.8087 -

*save Models*

In [25]:
model_simpleCNN.save('simpleCNN.h5')
# model_ResNet.save('ResNet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
source_file = 'simpleCNN.h5'

destination_folder = '/content/drive/My Drive/CUDA-dataset/Models'

shutil.move(source_file, destination_folder)

Error: Destination path '/content/drive/My Drive/CUDA-dataset/Models/simpleCNN.h5' already exists